In [19]:
import requests
import os
import cv2

Ready = True

ServerAPIURL = "https://server1.proyectohorus.com.ar"; # << modificar por la URL del servidor en formato http://.........../production
ServerUser = ""; # << modificar por el usuario creado en el administrador de la API
ServerPassword = ""; # << modificar por la clave creada en el adminisrador de la API
Profile = ""; # << escribir el UUID correspondiente al perfil creado en el administrador de la API, el mismo debe estar bajo el miosmo usuario arriba escrito

# A LA VARIABLE TOKEN VA A IR A VARIAI ESA MEDIA LLAVE QUE SE REQUIERE PARA INTERACTUAR CON EL SERVICIO
# EL TOKEN TIENE UNA DURACION DE 48HS, LUEGO SE DEBERA SOLICITAR UN NUEVO TOKE.

token = "";

def GetToken():
    global token
    global Ready
    
    #EN ESTA FUNCION OBTENGO EL TOKEN QUE ME PERMITIRA INTERACTUAR CON EL SERVICIO, ALGO ASI COMO EL USUARIO Y LA CONTRASEÑA
    
    if (token == ""):
        Ready = False
        
        # DEFINIMOS LAS VARIABLES DE LA FUNCION
        multipart_form_data = {
            'user': ServerUser,
            'password': ServerPassword,
            'profileuuid': Profile,
        }

        # ENVIO LOS DATOS AL SERVIDOR
        
        response = requests.post(ServerAPIURL + "/api/v2/functions/login", data=multipart_form_data)

        #COMO DEFINIMOS A outformat COMO pipe LA CADENA DE RETORNO SERA UN STRING SEPARADO POR "|", POR LO CUAL PARA OBTENER A CADA VALOR POR SEPARADO USAREMOS SPLIT('|'). 
                
        RecivedMatrix = response.text.split('|');

        if (RecivedMatrix[0] == "200"):
            token = RecivedMatrix[1]
        else:
            token = ""

        Ready = True
        
def Upload(ImageFile):
    #EN ESTA FUNCION HACEMOS EL UPLOAD DE LA IMAGEN A EVALUAR Y MUESTRA EL RESULTADO
    
    global token
    global Ready
    
    fromServer = ""

    try:
        # LO PRIMERO QUE HACEMOS ES CONSULTAR A LA VARIABLE READY PARA VER SI AUN HAY UNA RESPUESTA POR OBTENER DEL SERVIDOR ANTES DE ENVIAR UNA NUEVA CONSULTA      
        if (Ready == True):
            Ready = False

            # LLAMAMOS A LA FUNCION DE TOKEN
            GetToken()

            Recivedtmp = ""
            
            # DEFINIMOS LAS VARIABLES DE LA FUNCION
            img = cv2.imread(ImageFile)
            img_encoded = cv2.imencode('.jpg', img)[1]

            file = {
                'photo': ("face", img_encoded),
            }
            
            # ENVIO LOS DATOS AL SERVIDOR
            response = requests.post(ServerAPIURL + "/api/v2/functions/face/id", files=file, headers={"Authorization":"Bearer " + token})

            # RECIBIMOS LA RESPUESTA DESDE EL SERVIDOR
            Recivedtmp = response.text;
            
            # EL STRING NOS DEVOLVERA UN UUID COMO IDENTIFICACION DEL OBJETO O ROSTRO DETECTADO POR LO CUAL DEBEREMOS REMPLAZARLO POR LA INFORMACION CANONICA QUE DECLARAMOS EN EL ADMINISTRADOR DE LA API
            if (Recivedtmp.strip() != ""):
                # LO PRIMERO QUE HACEMOS ES SEPARAR LA RESPUESTA ES LINEAS YA QUE outformat LO DEFINIMOS COMO pipe, PUEDEN HABER MAS DE UNA RESPUESTA SI ES QUE EN LA IMAGEN HAY MAS DE UN OBJETO DETECTABLE (EJEMPLO: HAY DOS ROSTROS)
                Metadata = Recivedtmp.split('\n')

                # EVALUAMOS UNA A UNA LAS RESPUESTAS
                for Metaline in Metadata:
                    # SI LA LINEA NO ESTA VACIA IMPLICA QUE HAY UNA RESPUESTA POR PARTE DEL SERVIDOR
                    if (Metaline.strip() != ""):
                        # SEPARAMOS LAS LINEAS EN "|" YA QUE outformat LO DEFINIMOS COMO pipe
                        Values = Metaline.split('|');
                        
                        # ENVIO LOS DATOS AL SERVIDOR
                        response = requests.get(ServerAPIURL + "/api/v2/admin/accounts/users/profiles/detections=" + Values[6] + "/value", headers={"Authorization":"Bearer " + token})

                        # SEPARO LA RESPUESTA EN "|" YA QUE DEFINIMOS QUE outformat ESTA EN pipe
                        RecivedMatrix = response.text.split('|');

                        # SI EL CODIGO RECIBIDO EN LA POSICION O ES 200 SIGNIFICA QUE EL SERVIDOR RESPONDIO CORRECTAMENTE CON EL CANONICO DEL UUID POR LO CUAL PROCEDEMOS A REMPLAZARLO EN EL STRING DE DETECCION
                        if (RecivedMatrix[0] == "200"):
                            Recivedtmp = Recivedtmp.replace(Values[6], RecivedMatrix[1]);

                        print(RecivedMatrix)
                            
                        # YA CON TODO RECIBIDO Y FORMATEADO PASAMOS A INTERPRETAR LA INFORMACION      
                        ReceiveOnMatrix = Recivedtmp.split('|');

                        # COMO TENEMOS DEFINIDA LA RESPUESTA CON outformat EN pipe, CADA DETECCION SERA UN STREAM QUE OCUPARA UNA LINEA DONDE CADA RESPUESTA OCUPA UNA POSICION SEPARADA PO "|"
                        # EJEMPLO DEL STREAM EN FORMATO PIPE:
                        # [CODIGO DE ACCION]|[CANONICO DEL CODIGO]|[POSICION YMIN DEL BOX DE DETECCION]|[POSICION XMAX DEL BOX DE DETECCION]|[POSICION YMAX DEL BOX DE DETECCION]|[POSICION XMAX DEL BOX DE DETECCION]|[NOMBRE O UUID DE LO DETECTADO]|[UUID DEL GRUPO IRIS]|[CONFIDENCE DE LA DETECCION O SEA QUE TAN EXACTA ES LA DETECCION]
                        # EJEMPLO PRACTICO: 
                        #  200|ok|0.0|0.44375|0.47291666666666665|0.7140625|Juan Perez|1cc9a3d4461011ea9ca300155d016a1c|0.4329930749381952\n

                        
                        if (ReceiveOnMatrix[0] == "200"):
                            # LOS VALORES X e Y ESTAN SIN ESCALAR ESTO SIGNIFICA QUE DEBEREMOS MULTIPLICAR CADA VALOR POR EL ANCHO Y EL ALTO DE LA IMAGEN PARA OBTENER LAS COORDENADAS X/Y SOBRE LA IMAGEN 

                            ymin = float(ReceiveOnMatrix[2])
                            xmin = float(ReceiveOnMatrix[3])
                            ymax = float(ReceiveOnMatrix[4])
                            xmax = float(ReceiveOnMatrix[5])

                            left = int(xmin * 640)
                            right = int(xmax * 640)
                            top = int(ymin * 480)
                            bottom = int(ymax * 480)

                            # POR ULTIMO SEPRAMOS LOS DATOS Y LOS PRESENTAMOS
                            Name = ReceiveOnMatrix[6]
                            Confidence = float(ReceiveOnMatrix[8])

                            fromServer = "Left: " + str(left) + " | Right: " + str(right) + " | Top: " + str(top) + " | Left: " + str(left) + " | Name: " + Name + " | Confidence: " + str(Confidence);

            Ready = True    
            
    catch:
        Ready = True  
        
            
    return fromServer

# LLAMAMOS A LA FUNCION UPLOAD ESPECIFICANDO EL NOMBRE DEL ARCHIVO JPG A EVALUAR
print(Upload('archivo.jpg'))

['200', 'Fernando P. Maniglia']
Left: 184 | Right: 455 | Top: 65 | Left: 184 | Name: Fernando P. Maniglia | Confidence: 0.09206798695516005
